In [ ]:
import cv2
import numpy as np
import time



In [ ]:
#acceing camera
cap = cv2.VideoCapture(0)       #to access camera


In [ ]:
#capturing frames
a, background = cap.read()      #captures frames from webcam
time.sleep(3)                   #2 sec gap b/w capturing the background frames
a, background = cap.read()


In [ ]:
#define all the kernels size  
open_kernel = np.ones((20,20),np.uint8)
close_kernel = np.ones((20,20),np.uint8)
dilation_kernel = np.ones((20, 20), np.uint8)

In [ ]:
#removing noise
def filter_mask(mask):
    close_mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, close_kernel)
    open_mask = cv2.morphologyEx(close_mask, cv2.MORPH_OPEN, open_kernel)
    dilation = cv2.dilate(open_mask, dilation_kernel, iterations= 1)
    return dilation

In [ ]:
#masking
while cap.isOpened():
    a,frame=cap.read()
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_bound = np.array([50,80,50])
    upper_bound = np.array([90,255,255])
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    mask = filter_mask(mask)
    cloak = cv2.bitwise_and(background, background, mask=mask) #apply mask to take only those regions from the saved background 
                                                            #where our cloak is present in the current frame
    inverse_mask = cv2.bitwise_not(mask)
    current_background = cv2.bitwise_and(frame, frame, mask=inverse_mask)
    combined = cv2.add(cloak, current_background)
    
    cv2.imshow("Invisible Cloak", combined)


    if cv2.waitKey(1) == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
